## Descripción 

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

- Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región.
- Seleccionada: calidad de crudo y volumen de reservas.
- Crear un modelo para predecir el volumen de reservas en pozos nuevos.
- Elegir los pozos petrolíferos que tienen los valores estimados más altos.
- Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

### Descarga y preparación de los datos 

In [17]:
import pandas as pd
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt
import numpy as np

In [18]:
#Guardamos los archivos en la carpeta de trabajo
df_0 = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_0.csv")
df_0.to_csv("geo_data_0.csv",index = False)

df_1 = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_1.csv")
df_1.to_csv("geo_data_1.csv",index = False)

df_2 = pd.read_csv("https://practicum-content.s3.us-west-1.amazonaws.com/datasets/geo_data_2.csv")
df_2.to_csv("geo_data_2.csv",index = False)

In [19]:
#Cargamos los archivos
data_0 = pd.read_csv("geo_data_0.csv")
data_1 = pd.read_csv("geo_data_1.csv")
data_2 = pd.read_csv("geo_data_2.csv")

In [20]:
#Revisamos las tipos de variables en el dataset
print("Tipos de variables en el dataset geo_data_0:")
print(data_0.dtypes)
print (data_0.head(5))

Tipos de variables en el dataset geo_data_0:
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647


In [21]:
print("Tipos de variables en el dataset geo_data_1:")
print(data_1.dtypes)
print (data_1.head(5))

Tipos de variables en el dataset geo_data_1:
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305


In [22]:
print("Tipos de variables en el dataset geo_data_2:")
print(data_2.dtypes)
print (data_2.head(5))

Tipos de variables en el dataset geo_data_2:
id          object
f0         float64
f1         float64
f2         float64
product    float64
dtype: object
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


El campo Id es irelevante para el entrenamiento, por lo que lo eliminamos de los tres datasets

In [23]:
data_0.drop(columns=['id'], inplace=True)
data_1.drop(columns=['id'], inplace=True)
data_2.drop(columns=['id'], inplace=True)

### Entrena y prueba el modelo para cada región

Definimos la función para entrenar los modelos 

In [24]:
def train_and_prove_model(data):
    features = data.drop('product',axis=1)
    target = data['product']
    # separamos los datos en entrenamiento y validación 75% y 25%
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=54321)
    # usamos la regresión lineal como modelo
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    # calcula la RECM en el conjunto de validació
    result = mean_squared_error(target_valid, predictions_valid) ** 0.5 
    print("Volumen medio de reservas:", predictions_valid.mean())
    print("RECM del modelo:", result)
    return predictions_valid

In [25]:
# Entrenamos y validamos los data sets
print("Data set geo_data_0:")
prediction_geo_0 = train_and_prove_model(data_0)
print("-" * 50)
print("Data set geo_data_1:")
prediction_geo_1 = train_and_prove_model(data_1)
print("-" * 50)
print("Data set geo_data_2:")
prediction_geo_2 = train_and_prove_model(data_2)
print("-" * 50)

Data set geo_data_0:
Volumen medio de reservas: 92.15921155743655
RECM del modelo: 37.68341093860808
--------------------------------------------------
Data set geo_data_1:
Volumen medio de reservas: 68.44594093153303
RECM del modelo: 0.8922819708481438
--------------------------------------------------
Data set geo_data_2:
Volumen medio de reservas: 94.92229500787454
RECM del modelo: 40.15250541784855
--------------------------------------------------


### Preparación para el cálculo de ganancias 

In [26]:
inversion = 100000000
pozos = 200
produccion_esperada_por_pozo = 500000
unidades_esperadas_por_pozo = 111.1
precio_por_barril = 4.5

Función para calcular ganancias de un conjunto de pozos

In [27]:
def calculate_profitability(data,pozos, precio_por_barril, inversion):
    # obtener las 200 predicciones más altas
    predicciones = data.nlargest(pozos, 'product')
    predicciones['ganancias'] = predicciones['product'] * 1000 * precio_por_barril
    total_ganancias = predicciones['ganancias'].sum()
    print("Ganancias totales:", total_ganancias)
    print("Promedio de unidades por pozo:", predicciones['product'].mean())
    print("Rendimiento sobre la inversión:", (total_ganancias - inversion) / inversion * 100, "%")
    return predicciones


Calculamos las ganacias para cada region

In [28]:
print("Data set geo_data_0:")
prediciones_geo_0 = calculate_profitability(data_0,pozos, precio_por_barril, inversion)
print("-" * 50)
print("Data set geo_data_1:")
prediciones_geo_1 = calculate_profitability(data_1,pozos, precio_por_barril, inversion)
print("-" * 50)
print("Data set geo_data_2:")
prediciones_geo_2 = calculate_profitability(data_2,pozos, precio_por_barril, inversion)
print("-" * 50)

Data set geo_data_0:
Ganancias totales: 166350365.6808241
Promedio de unidades por pozo: 184.83373964536008
Rendimiento sobre la inversión: 66.3503656808241 %
--------------------------------------------------
Data set geo_data_1:
Ganancias totales: 124150866.96681508
Promedio de unidades por pozo: 137.9454077409057
Rendimiento sobre la inversión: 24.150866966815084 %
--------------------------------------------------
Data set geo_data_2:
Ganancias totales: 170596329.2835898
Promedio de unidades por pozo: 189.55147698176646
Rendimiento sobre la inversión: 70.59632928358981 %
--------------------------------------------------


La tercera región parece ser las mas conveniente para el desarrollo de los posos petroleros, ya que tenemos un mayor porcentaje en el rendimineto de la inversión. Además al entrenar los modelos tuvimos el RECM mas alto, por lo que las precciónes con las que estamos trabajando parecen ser las mas confiables 

### Calcular riesgos y ganancias

In [29]:
def calculate_risk_profit(data):
    state = np.random.RandomState(12345) 
    values = []
    # Aplicamos Boostrapping para 1000 muestras
    for i in range(1000):
        #Se coloca replace=True para que el mismo elemento pueda caer en diferentes muestras
        subsample = data.sample(frac=25, replace=True, random_state=state) 
        #Calculamos el percentil 99
        values.append(subsample.quantile(0.99))

    values = pd.Series(values)
    veneficio_promedio = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    print("Beneficio promedio:", veneficio_promedio)
    print("Intervalo de confianza del 95%:", lower, upper)

In [30]:
print("prediciones_geo_0:")
calculate_risk_profit(prediciones_geo_0['product'])
print("-" * 50)
print("prediciones_geo_1:")
calculate_risk_profit(prediciones_geo_1['product'])
print("-" * 50)
print("prediciones_geo_2:")
calculate_risk_profit(prediciones_geo_2['product'])
print("-" * 50)

prediciones_geo_0:
Beneficio promedio: 185.35884389187544
Intervalo de confianza del 95%: 185.35561503190544 185.3626902521712
--------------------------------------------------
prediciones_geo_1:
Beneficio promedio: 137.94540774090567
Intervalo de confianza del 95%: 137.94540774090564 137.94540774090564
--------------------------------------------------
prediciones_geo_2:
Beneficio promedio: 190.0125725279819
Intervalo de confianza del 95%: 190.01172239678743 190.01358899804384
--------------------------------------------------


La tercera región sigue pareciendo la mejor opción para el desarrollo ya que tenemos un beneficio promedio mas alto. 